In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()

     |████████████████████████████████| 911kB 5.0MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
import os
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys

In [ ]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument("window-size=1200x600")
driver = webdriver.Chrome('chromedriver',options=chrome_options)

Открывается страница судебной практики Московского Арбитражного суда в "КонсультантПлюс"

In [ ]:
driver.get("http://www.consultant.ru/cons/cgi/online.cgi?rnd=4AAB9B02777E1C04F3746779D2B7EBE4&req=card&div=ARB#div")

In [ ]:
driver.get('http://www.consultant.ru/cons/cgi/online.cgi?req=query&cacheid=C398BC4FCCB680025605802AE8EF2FB2&NOQUERYLOG=1&ts=862933004204088034180744&sb=24&from=24&rnd=4AAB9B02777E1C04F3746779D2B7EBE4')

In [ ]:
data = {}

In [ ]:
list_page = driver.window_handles[0]

Основной цикл с процессом парсинга: открывается вкладка с документном, нажимается кнопка для его скачивания в формате rtf, для удобства скачанный документ переименовывается в соответствии с индексом, парсится окно "справка", где содержится полное название и аннотация документа:

In [ ]:
import time

# числа в range - начальная и конечная позиции документов в общей поисковой выдаче системы
# поскольку код запускался в несколько заходов и извлеченные данные сохранялись частями, то нумерация идет не с нуля
# в этой ячейке - последний запускавшийся цикл (до последнего документа от 15 июля)
for x in tqdm(range(18868, 22000)):
  print(x)
  try:
    driver.find_element_by_css_selector('body').send_keys(Keys.DOWN)
    driver.find_element_by_xpath('//div[@class="row current"]').find_element_by_tag_name('a').click()
    
    doc_page = driver.window_handles[1]
    driver.switch_to.window(doc_page)

    driver.implicitly_wait(1)
    driver.find_element_by_xpath('//button[@class="word"]').click()

    fileends = "crdownload"
    while "crdownload" == fileends:
      time.sleep(1)
      newest_file = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)[-1]
      if "crdownload" in newest_file:
        fileends = "crdownload"
      else:
        fileends = "none"  

    print(newest_file)

    if (re.search(r'\d+\.txt', newest_file)) or (re.search(r'.ipynb_checkpoints', newest_file)):
      if driver.current_window_handle!=list_page:
        driver.close()
        driver.switch_to.window(list_page)
      continue

    else:
      newname = str(x)+'.txt'
      os.rename(newest_file, newname)
      
      driver.find_element_by_xpath('//button[@class="esse"]').click()
      bs = BeautifulSoup(driver.page_source, 'html.parser')
      spans = bs.find('div',attrs={'class':'zone'}).find_all('span')
      for i, s in enumerate(spans):
        if s.string == 'Название документа':
          start_index = i+1
      title = spans[start_index].string
      summary = '\n'.join([s.text for s in spans[start_index+1:]])


      data[x] = {'title':title, 'summary': summary}


    driver.close()
    driver.switch_to.window(list_page)


  except Exception as e:
    print(e)
    if driver.current_window_handle!=list_page:
      driver.close()
      driver.switch_to.window(list_page)

Выходные данные были обрезаны до нескольких последних строк (5000).
Постановление Арбитражного суда Московского округа от 07.08..rtf
19501
Постановление Арбитражного суда Московского округа от 07.08..rtf
19502
Постановление Арбитражного суда Московского округа от 07.08..rtf
19503
Постановление Арбитражного суда Московского округа от 07.08..rtf
19504
Постановление Арбитражного суда Московского округа от 07.08..rtf
19505
Постановление Арбитражного суда Московского округа от 07.08..rtf
19506
Постановление Арбитражного суда Московского округа от 07.08..rtf
19507
Постановление Арбитражного суда Московского округа от 06.08..rtf
19508
Постановление Арбитражного суда Московского округа от 06.08..rtf
19509
Постановление Арбитражного суда Московского округа от 06.08..rtf
19510
Постановление Арбитражного суда Московского округа от 06.08..rtf
19511
Постановление Арбитражного суда Московского округа от 06.08..rtf
19512
Постановление Арбитражного суда Московского округа от 06.08..rtf
19513
Постановл

In [ ]:
data[20000]

{'summary': 'Требование: О взыскании неосновательного обогащения, неустойки.\nОбстоятельства: Объем полезного отпуска по потребителю за спорный период был сформирован некорректно.\nРешение: Требование удовлетворено частично, поскольку факт обогащения исполнителя за счет заказчика подтвержден документами, свидетельствующими об оплате оказанных услуг в завышенном размере, платежными поручениями, свидетельствующими о получении денежных средств ответчиком в большем размере (переплате). Размер неустойки снижен с учетом ставки рефинансирования Центрального банка РФ на дату принятия резолютивной части решения суда.',
 'title': 'Постановление Арбитражного суда Московского округа от 31.07.2020 N Ф05-9203/2020 по делу N А40-258148/2019'}

Функция для извлечения "сырого" текста из формата rtf взята отсюда: https://stackoverflow.com/questions/188545/regular-expression-for-extracting-text-from-an-rtf-string/188877#188877

In [1]:
def striprtf(text):
   pattern = re.compile(r"\\([a-z]{1,32})(-?\d{1,10})?[ ]?|\\'([0-9a-f]{2})|\\([^a-z])|([{}])|[\r\n]+|(.)", re.I)
   # control words which specify a "destionation".
   destinations = frozenset((
      'aftncn','aftnsep','aftnsepc','annotation','atnauthor','atndate','atnicn','atnid',
      'atnparent','atnref','atntime','atrfend','atrfstart','author','background',
      'bkmkend','bkmkstart','blipuid','buptim','category','colorschememapping',
      'colortbl','comment','company','creatim','datafield','datastore','defchp','defpap',
      'do','doccomm','docvar','dptxbxtext','ebcend','ebcstart','factoidname','falt',
      'fchars','ffdeftext','ffentrymcr','ffexitmcr','ffformat','ffhelptext','ffl',
      'ffname','ffstattext','field','file','filetbl','fldinst','fldrslt','fldtype',
      'fname','fontemb','fontfile','fonttbl','footer','footerf','footerl','footerr',
      'footnote','formfield','ftncn','ftnsep','ftnsepc','g','generator','gridtbl',
      'header','headerf','headerl','headerr','hl','hlfr','hlinkbase','hlloc','hlsrc',
      'hsv','htmltag','info','keycode','keywords','latentstyles','lchars','levelnumbers',
      'leveltext','lfolevel','linkval','list','listlevel','listname','listoverride',
      'listoverridetable','listpicture','liststylename','listtable','listtext',
      'lsdlockedexcept','macc','maccPr','mailmerge','maln','malnScr','manager','margPr',
      'mbar','mbarPr','mbaseJc','mbegChr','mborderBox','mborderBoxPr','mbox','mboxPr',
      'mchr','mcount','mctrlPr','md','mdeg','mdegHide','mden','mdiff','mdPr','me',
      'mendChr','meqArr','meqArrPr','mf','mfName','mfPr','mfunc','mfuncPr','mgroupChr',
      'mgroupChrPr','mgrow','mhideBot','mhideLeft','mhideRight','mhideTop','mhtmltag',
      'mlim','mlimloc','mlimlow','mlimlowPr','mlimupp','mlimuppPr','mm','mmaddfieldname',
      'mmath','mmathPict','mmathPr','mmaxdist','mmc','mmcJc','mmconnectstr',
      'mmconnectstrdata','mmcPr','mmcs','mmdatasource','mmheadersource','mmmailsubject',
      'mmodso','mmodsofilter','mmodsofldmpdata','mmodsomappedname','mmodsoname',
      'mmodsorecipdata','mmodsosort','mmodsosrc','mmodsotable','mmodsoudl',
      'mmodsoudldata','mmodsouniquetag','mmPr','mmquery','mmr','mnary','mnaryPr',
      'mnoBreak','mnum','mobjDist','moMath','moMathPara','moMathParaPr','mopEmu',
      'mphant','mphantPr','mplcHide','mpos','mr','mrad','mradPr','mrPr','msepChr',
      'mshow','mshp','msPre','msPrePr','msSub','msSubPr','msSubSup','msSubSupPr','msSup',
      'msSupPr','mstrikeBLTR','mstrikeH','mstrikeTLBR','mstrikeV','msub','msubHide',
      'msup','msupHide','mtransp','mtype','mvertJc','mvfmf','mvfml','mvtof','mvtol',
      'mzeroAsc','mzeroDesc','mzeroWid','nesttableprops','nextfile','nonesttables',
      'objalias','objclass','objdata','object','objname','objsect','objtime','oldcprops',
      'oldpprops','oldsprops','oldtprops','oleclsid','operator','panose','password',
      'passwordhash','pgp','pgptbl','picprop','pict','pn','pnseclvl','pntext','pntxta',
      'pntxtb','printim','private','propname','protend','protstart','protusertbl','pxe',
      'result','revtbl','revtim','rsidtbl','rxe','shp','shpgrp','shpinst',
      'shppict','shprslt','shptxt','sn','sp','staticval','stylesheet','subject','sv',
      'svb','tc','template','themedata','title','txe','ud','upr','userprops',
      'wgrffmtfilter','windowcaption','writereservation','writereservhash','xe','xform',
      'xmlattrname','xmlattrvalue','xmlclose','xmlname','xmlnstbl',
      'xmlopen',
   ))
   # Translation of some special characters.
   specialchars = {
      'par': '\n',
      'sect': '\n\n',
      'page': '\n\n',
      'line': '\n',
      'tab': '\t',
      'emdash': u'\u2014',
      'endash': u'\u2013',
      'emspace': u'\u2003',
      'enspace': u'\u2002',
      'qmspace': u'\u2005',
      'bullet': u'\u2022',
      'lquote': u'\u2018',
      'rquote': u'\u2019',
      'ldblquote': u'\201C',
      'rdblquote': u'\u201D', 
   }
   stack = []
   ignorable = False       # Whether this group (and all inside it) are "ignorable".
   ucskip = 1              # Number of ASCII characters to skip after a unicode character.
   curskip = 0             # Number of ASCII characters left to skip
   out = []                # Output buffer.
   for match in pattern.finditer(text):
      word,arg,hex,char,brace,tchar = match.groups()
      if brace:
         curskip = 0
         if brace == '{':
            # Push state
            stack.append((ucskip,ignorable))
         elif brace == '}':
            # Pop state
            ucskip,ignorable = stack.pop()
      elif char: # \x (not a letter)
         curskip = 0
         if char == '~':
            if not ignorable:
                out.append(u'\xA0')
         elif char in '{}\\':
            if not ignorable:
               out.append(char)
         elif char == '*':
            ignorable = True
      elif word: # \foo
         curskip = 0
         if word in destinations:
            ignorable = True
         elif ignorable:
            pass
         elif word in specialchars:
            out.append(specialchars[word])
         elif word == 'uc':
            ucskip = int(arg)
         elif word == 'u':
            c = int(arg)
            if c < 0: c += 0x10000
            if c > 127: out.append(unichr(c))
            else: out.append(chr(c))
            curskip = ucskip
      elif hex: # \'xx
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            c = int(hex,16)
            if c > 127: out.append(unichr(c))
            else: out.append(chr(c))
      elif tchar:
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            out.append(tchar)
   return ''.join(out)

Завершение работы виртуального браузера:

In [ ]:
driver.quit()

Добавление содержимого документов в общую переменную data

In [ ]:
filelist = [f for f in os.listdir() if f.endswith('.txt')]

for index in data.keys():
  docname = str(index)+'.txt'
  try:
    with open(docname, encoding="cp1251") as fil:
      doc = fil.read()
    data[index]['text'] = striprtf(doc)
  except:
    data[index]['text'] = ''

In [ ]:
data[21999]

{'summary': 'Требование: О взыскании неосновательного обогащения.\nОбстоятельства: Лизингополучателем была допущена просрочка при внесении лизинговых платежей, лизингодатель направил уведомление об отказе от исполнения договора лизинга в одностороннем порядке, с требованием о возврате имущества и оплаты задолженности.\nВстречное требование: О взыскании задолженности по договору лизинга.\nРешение: В удовлетворении первоначального иска отказано и встречный иск удовлетворен, поскольку лизингополучателем была допущена просрочка при внесении лизинговых платежей; действуя по своей воле и в своем интересе, лизингополучатель принял условия договора без замечаний, согласившись с положениями, касающимися определения завершающей обязанности по сделке.',
 'text': '\nАРБИТРАЖНЫЙ СУД МОСКОВСКОГО ОКРУГА\n\nПОСТАНОВЛЕНИЕ\nот 15 июля 2020 г. по делу N А40-145387/2019\n\nРезолютивная часть постановления объявлена 08.07.2020\nПолный текст постановления изготовлен 15.07.2020\nАрбитражный суд Московского о

Сохранение словаря в json:

In [ ]:
import json

with open('moscow_court18868-21999.json', 'w') as f:
  json.dump(data, f, ensure_ascii=False)

In [ ]:
from google.colab import files

files.download('moscow_court18868-21999.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>